<a href="https://colab.research.google.com/github/abhish3kSW/python/blob/master/truedatamonthwise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# INSTALLATION
# !pip install truedata-ws pandas openpyxl nsepython requests

import pandas as pd
import time
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from google.colab import files
from truedata_ws.websocket.TD import TD
import calendar
import requests
from io import StringIO

# ============================================================================
# CONFIGURATION
# ============================================================================
USERNAME = 'tdwsf550'
PASSWORD = 'parang@550'

# Symbols to fetch
ROOT_SYMBOLS = ['NIFTY', 'MIDCPNIFTY']

# Date Range
START_DATE = datetime(2024, 1, 1)
END_DATE = datetime(2026, 1, 1)

# ============================================================================
# EXPIRY CALENDAR - COMPREHENSIVE NSE HISTORY
# ============================================================================

# NSE Expiry Rules:
# - NIFTY, BANKNIFTY: Last Thursday (changed to Wednesday in Nov 2024)
# - FINNIFTY: Last Tuesday
# - MIDCPNIFTY: Last Monday

EXPIRY_CALENDARS = {
    # Format: 'YYYY-MM': 'YYYY-MM-DD'
    # NIFTY & BANKNIFTY (Thursday → Wednesday from Nov 2024)
    'NIFTY': {
        '2024-01': '2024-01-25', '2024-02': '2024-02-29', '2024-03': '2024-03-28',
        '2024-04': '2024-04-25', '2024-05': '2024-05-30', '2024-06': '2024-06-27',
        '2024-07': '2024-07-25', '2024-08': '2024-08-29', '2024-09': '2024-09-26',
        '2024-10': '2024-10-31', '2024-11': '2024-11-27', '2024-12': '2024-12-25',  # Wed from Nov
        '2025-01': '2025-01-29', '2025-02': '2025-02-26', '2025-03': '2025-03-26',
        '2025-04': '2025-04-30', '2025-05': '2025-05-28', '2025-06': '2025-06-25',
        '2025-07': '2025-07-30', '2025-08': '2025-08-27', '2025-09': '2025-09-24',
        '2025-10': '2025-10-29', '2025-11': '2025-11-26', '2025-12': '2025-12-24',
    },

    'BANKNIFTY': {
        '2024-01': '2024-01-25', '2024-02': '2024-02-29', '2024-03': '2024-03-28',
        '2024-04': '2024-04-25', '2024-05': '2024-05-30', '2024-06': '2024-06-27',
        '2024-07': '2024-07-25', '2024-08': '2024-08-29', '2024-09': '2024-09-26',
        '2024-10': '2024-10-31', '2024-11': '2024-11-27', '2024-12': '2024-12-25',
        '2025-01': '2025-01-29', '2025-02': '2025-02-26', '2025-03': '2025-03-26',
        '2025-04': '2025-04-30', '2025-05': '2025-05-28', '2025-06': '2025-06-25',
        '2025-07': '2025-07-30', '2025-08': '2025-08-27', '2025-09': '2025-09-24',
        '2025-10': '2025-10-29', '2025-11': '2025-11-26', '2025-12': '2025-12-24',
    },

    # FINNIFTY - Last Tuesday
    'FINNIFTY': {
        '2024-01': '2024-01-30', '2024-02': '2024-02-27', '2024-03': '2024-03-26',
        '2024-04': '2024-04-30', '2024-05': '2024-05-28', '2024-06': '2024-06-25',
        '2024-07': '2024-07-30', '2024-08': '2024-08-27', '2024-09': '2024-09-24',
        '2024-10': '2024-10-29', '2024-11': '2024-11-26', '2024-12': '2024-12-31',
        '2025-01': '2025-01-28', '2025-02': '2025-02-25', '2025-03': '2025-03-25',
        '2025-04': '2025-04-29', '2025-05': '2025-05-27', '2025-06': '2025-06-24',
        '2025-07': '2025-07-29', '2025-08': '2025-08-26', '2025-09': '2025-09-30',
        '2025-10': '2025-10-28', '2025-11': '2025-11-25', '2025-12': '2025-12-30',
    },

    # MIDCPNIFTY - Last Monday
    'MIDCPNIFTY': {
        '2024-01': '2024-01-29', '2024-02': '2024-02-26', '2024-03': '2024-03-25',
        '2024-04': '2024-04-29', '2024-05': '2024-05-27', '2024-06': '2024-06-24',
        '2024-07': '2024-07-29', '2024-08': '2024-08-26', '2024-09': '2024-09-30',
        '2024-10': '2024-10-28', '2024-11': '2024-11-25', '2024-12': '2024-12-30',
        '2025-01': '2025-01-27', '2025-02': '2025-02-24', '2025-03': '2025-03-31',
        '2025-04': '2025-04-28', '2025-05': '2025-05-26', '2025-06': '2025-06-30',
        '2025-07': '2025-07-28', '2025-08': '2025-08-25', '2025-09': '2025-09-29',
        '2025-10': '2025-10-27', '2025-11': '2025-11-24', '2025-12': '2025-12-29',
    },
}

def get_expiry_date(root_symbol, year, month):
    """
    Get actual expiry date from calendar or calculate fallback.
    """
    month_key = f"{year}-{month:02d}"

    # Try to get from calendar
    if root_symbol in EXPIRY_CALENDARS:
        if month_key in EXPIRY_CALENDARS[root_symbol]:
            return datetime.strptime(EXPIRY_CALENDARS[root_symbol][month_key], '%Y-%m-%d')

    # Fallback: Calculate based on typical rules
    print(f"   ⚠️  Using calculated expiry for {root_symbol} {month_key}")
    return calculate_expiry_fallback(root_symbol, year, month)

def calculate_expiry_fallback(root_symbol, year, month):
    """
    Calculate expiry as fallback (last Thursday/Tuesday/Monday).
    """
    last_day = calendar.monthrange(year, month)[1]
    last_date = datetime(year, month, last_day)

    # Determine target weekday
    if root_symbol in ['NIFTY', 'BANKNIFTY']:
        target_weekday = 3  # Thursday (Wednesday from Nov 2024)
        if year == 2024 and month >= 11:
            target_weekday = 2  # Wednesday
        elif year >= 2025:
            target_weekday = 2  # Wednesday
    elif root_symbol == 'FINNIFTY':
        target_weekday = 1  # Tuesday
    elif root_symbol == 'MIDCPNIFTY':
        target_weekday = 0  # Monday
    else:
        target_weekday = 3  # Default Thursday

    offset = (last_date.weekday() - target_weekday) % 7
    expiry = last_date - timedelta(days=offset)
    return expiry

# ============================================================================
# CONTRACT GENERATION WITH SMART DATE WINDOWS
# ============================================================================

def get_contract_info(root_symbol, year, month):
    """
    Generate contract info with smart date windows.
    Starts from last week of previous month to catch rollover.
    """
    # Get actual expiry
    expiry = get_expiry_date(root_symbol, year, month)

    # Contract symbol format
    month_code = expiry.strftime('%b').upper()
    year_code = expiry.strftime('%y')
    contract_symbol = f"{root_symbol}{year_code}{month_code}FUT"

    # Start date: Last week of PREVIOUS month (7 days before month start)
    month_start = datetime(year, month, 1)
    start_date = month_start - timedelta(days=7)

    # End date: 3 days AFTER expiry to catch settlement
    end_date = expiry + timedelta(days=3)

    return {
        'symbol': contract_symbol,
        'start_date': start_date,
        'end_date': end_date,
        'expiry_date': expiry,
        'year': year,
        'month': month,
        'month_name': calendar.month_name[month]
    }

def generate_all_contracts(root_symbol, start_date, end_date):
    """
    Generate all contracts between date range.
    """
    contracts = []
    current = start_date.replace(day=1)

    while current < end_date:
        contract = get_contract_info(root_symbol, current.year, current.month)

        # Include if expiry is after our start date
        if contract['expiry_date'] >= start_date:
            contracts.append(contract)

        current += relativedelta(months=1)

    return contracts

# ============================================================================
# SYMBOL FORMAT DETECTION
# ============================================================================

def detect_symbol_format(td_obj, root_symbol, test_date=datetime(2024, 1, 1)):
    """
    Auto-detect the correct symbol format by testing variants.
    """
    year = test_date.year
    month = test_date.month
    month_code = test_date.strftime('%b').upper()
    year_code = test_date.strftime('%y')

    variants = [
        f"{root_symbol}{year_code}{month_code}FUT",  # NIFTY24JANFUT
        f"{root_symbol}{year_code}{month_code}",      # NIFTY24JAN
        f"{root_symbol} {year_code}{month_code}FUT",  # NIFTY 24JANFUT
        f"{root_symbol}-{month_code}{year_code}FUT",  # NIFTY-JAN24FUT
    ]

    print(f"\n🔍 Auto-detecting symbol format for {root_symbol}...")

    for variant in variants:
        try:
            # Try fetching 1 day of data as test
            test_start = datetime(year, month, 15)
            data = td_obj.get_historic_data(
                variant,
                start_time=test_start,
                bar_size='1 day'
            )

            if data and len(data) > 0:
                print(f"   ✅ FOUND: {variant}")
                return variant.replace(f"{year_code}{month_code}FUT", "{yy}{mon}FUT")
        except:
            continue

    print(f"   ⚠️  Using default format")
    return f"{root_symbol}{{yy}}{{mon}}FUT"

# ============================================================================
# DATA FETCHING
# ============================================================================

def fetch_contract_data(td_obj, contract_info, symbol_format, bar_size='5 mins'):
    """
    Fetch data for a single contract with the detected format.
    """
    # Format symbol
    yy = contract_info['expiry_date'].strftime('%y')
    mon = contract_info['expiry_date'].strftime('%b').upper()
    symbol = symbol_format.replace('{yy}', yy).replace('{mon}', mon)

    start = contract_info['start_date']
    end = contract_info['end_date']

    print(f"\n📊 {symbol}")
    print(f"   Fetch Window: {start.date()} → {end.date()}")
    print(f"   Expiry: {contract_info['expiry_date'].date()} ({contract_info['expiry_date'].strftime('%A')})")

    try:
        raw_data = td_obj.get_historic_data(
            symbol,
            start_time=start,
            end_time=end,
            bar_size=bar_size
        )

        if raw_data and len(raw_data) > 0:
            df = pd.DataFrame(raw_data)

            # Debug: Print columns received
            print(f"   📋 Columns received: {list(df.columns)}")

            # Standardize column names (TrueData might use different cases)
            df.columns = df.columns.str.lower()

            # Check if we have required columns
            if 'time' not in df.columns:
                print(f"   ❌ Missing 'time' column")
                return None

            # Add metadata
            df['contract'] = symbol
            df['expiry_date'] = contract_info['expiry_date']
            df['month_name'] = contract_info['month_name']
            df['time'] = pd.to_datetime(df['time'])

            # Clean data within window
            df = df[(df['time'] >= start) & (df['time'] <= end)]

            if not df.empty:
                print(f"   ✅ {len(df):,} bars | {df['time'].min().date()} to {df['time'].max().date()}")
                return df
            else:
                print(f"   ⚠️  Data filtered out")
                return None
        else:
            print(f"   ❌ No data returned")
            return None

    except Exception as e:
        print(f"   ❌ Error: {str(e)[:80]}")
        import traceback
        print(f"   Full error: {traceback.format_exc()}")
        return None

# ============================================================================
# MAIN EXECUTION
# ============================================================================

def main():
    print("="*80)
    print(" ADVANCED TRUEDATA FUTURES FETCHER WITH NSE EXPIRY CALENDAR")
    print("="*80)

    # Connect
    print("\n🔌 Connecting to TrueData...")
    try:
        td_obj = TD(USERNAME, PASSWORD, live_port=None)
        print("✅ Connected!\n")
    except Exception as e:
        print(f"❌ Connection failed: {e}")
        return

    # Process each symbol
    for root in ROOT_SYMBOLS:
        print(f"\n{'='*80}")
        print(f" {root}")
        print(f"{'='*80}")

        # Auto-detect symbol format
        symbol_format = detect_symbol_format(td_obj, root)

        # Generate contracts
        contracts = generate_all_contracts(root, START_DATE, END_DATE)
        print(f"\n📅 {len(contracts)} contracts scheduled")

        # Show calendar
        print("\n" + "-"*80)
        print(f"{'#':<4} {'Contract':<20} {'Expiry Date':<15} {'Day':<10} {'Fetch Period'}")
        print("-"*80)
        for i, c in enumerate(contracts, 1):
            day_name = c['expiry_date'].strftime('%A')
            fetch_period = f"{c['start_date'].date()} to {c['end_date'].date()}"
            print(f"{i:<4} {c['symbol']:<20} {c['expiry_date'].date()!s:<15} {day_name:<10} {fetch_period}")
        print("-"*80)

        # Fetch data
        all_dfs = []
        failed = []

        for i, contract in enumerate(contracts, 1):
            print(f"\n[{i}/{len(contracts)}]", end=" ")

            df = fetch_contract_data(td_obj, contract, symbol_format, bar_size='5 mins')

            if df is not None and not df.empty:
                all_dfs.append(df)
            else:
                failed.append(contract['symbol'])

            time.sleep(0.3)  # Rate limit

        # Save results
        if all_dfs:
            print(f"\n{'='*80}")
            print(" SAVING DATA")
            print(f"{'='*80}")

            combined = pd.concat(all_dfs, ignore_index=True)
            combined = combined.sort_values('time').reset_index(drop=True)

            # Add derived columns
            combined['root_symbol'] = root
            combined['date'] = combined['time'].dt.date
            combined['days_to_expiry'] = (combined['expiry_date'] - combined['time']).dt.days

            # Stats
            print(f"\n📊 STATISTICS:")
            print(f"   Total Bars: {len(combined):,}")
            print(f"   Date Range: {combined['time'].min()} → {combined['time'].max()}")
            print(f"   Contracts: {combined['contract'].nunique()}")
            print(f"   Success: {len(all_dfs)}/{len(contracts)}")
            print(f"   Columns: {list(combined.columns)}")

            if failed:
                print(f"\n   ⚠️  Failed: {', '.join(failed)}")

            # Save CSV
            csv_file = f"{root}_Futures_2024_2026_Complete.csv"
            combined.to_csv(csv_file, index=False)
            print(f"\n💾 Saved: {csv_file}")
            files.download(csv_file)

            # Save Excel with analysis (only if standard columns exist)
            excel_file = f"{root}_Futures_2024_2026_Analysis.xlsx"
            with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
                # All data
                combined.to_excel(writer, sheet_name='All Data', index=False)

                # Only create summary sheets if we have the right columns
                if 'close' in combined.columns and 'volume' in combined.columns:
                    # Summary by contract
                    summary = combined.groupby(['contract', 'expiry_date', 'month_name']).agg({
                        'time': ['min', 'max', 'count'],
                        'volume': 'sum',
                        'close': ['first', 'last', 'min', 'max']
                    }).round(2)
                    summary.to_excel(writer, sheet_name='Contract Summary')

                    # Daily stats
                    daily = combined.groupby('date').agg({
                        'volume': 'sum',
                        'close': ['min', 'max', 'mean']
                    }).round(2)
                    daily.to_excel(writer, sheet_name='Daily Summary')
                else:
                    # Create basic summary
                    summary = combined.groupby('contract').agg({
                        'time': ['min', 'max', 'count']
                    })
                    summary.to_excel(writer, sheet_name='Summary')

            print(f"💾 Saved: {excel_file}")
            files.download(excel_file)

        else:
            print(f"\n❌ NO DATA FOR {root}")
            print("   Possible issues:")
            print("   - Check TrueData plan limits")
            print("   - Verify symbol format")
            print("   - Confirm historical data access")

    print(f"\n{'='*80}")
    print(" ✅ COMPLETE!")
    print(f"{'='*80}")

if __name__ == "__main__":
    main()

 ADVANCED TRUEDATA FUTURES FETCHER WITH NSE EXPIRY CALENDAR

🔌 Connecting to TrueData...


(2026-01-16 08:28:32,702) WARNING :: Connected successfully to TrueData Historical Data Service...  (PID:385 Thread:133756759560192)
(2026-01-16 08:28:32,702) WARNING :: Connected successfully to TrueData Historical Data Service...  (PID:385 Thread:133756759560192)
(2026-01-16 08:28:32,702) WARNING :: Connected successfully to TrueData Historical Data Service...  (PID:385 Thread:133756759560192)
(2026-01-16 08:28:32,702) WARNING :: Connected successfully to TrueData Historical Data Service...  (PID:385 Thread:133756759560192)
(2026-01-16 08:28:32,702) WARNING :: Connected successfully to TrueData Historical Data Service...  (PID:385 Thread:133756759560192)


✅ Connected!


 NIFTY

🔍 Auto-detecting symbol format for NIFTY...


(2026-01-16 08:28:33,134) ERROR :: <class '_block.LZ4BlockError'> -> Decompression failed: corrupt input or insufficient space in destination buffer. Error code: 2 (PID:385 Thread:133756759560192)
(2026-01-16 08:28:33,134) ERROR :: <class '_block.LZ4BlockError'> -> Decompression failed: corrupt input or insufficient space in destination buffer. Error code: 2 (PID:385 Thread:133756759560192)
(2026-01-16 08:28:33,134) ERROR :: <class '_block.LZ4BlockError'> -> Decompression failed: corrupt input or insufficient space in destination buffer. Error code: 2 (PID:385 Thread:133756759560192)
(2026-01-16 08:28:33,134) ERROR :: <class '_block.LZ4BlockError'> -> Decompression failed: corrupt input or insufficient space in destination buffer. Error code: 2 (PID:385 Thread:133756759560192)
(2026-01-16 08:28:33,134) ERROR :: <class '_block.LZ4BlockError'> -> Decompression failed: corrupt input or insufficient space in destination buffer. Error code: 2 (PID:385 Thread:133756759560192)
ERROR:truedata_

   ⚠️  Using default format

📅 24 contracts scheduled

--------------------------------------------------------------------------------
#    Contract             Expiry Date     Day        Fetch Period
--------------------------------------------------------------------------------
1    NIFTY24JANFUT        2024-01-25      Thursday   2023-12-25 to 2024-01-28
2    NIFTY24FEBFUT        2024-02-29      Thursday   2024-01-25 to 2024-03-03
3    NIFTY24MARFUT        2024-03-28      Thursday   2024-02-23 to 2024-03-31
4    NIFTY24APRFUT        2024-04-25      Thursday   2024-03-25 to 2024-04-28
5    NIFTY24MAYFUT        2024-05-30      Thursday   2024-04-24 to 2024-06-02
6    NIFTY24JUNFUT        2024-06-27      Thursday   2024-05-25 to 2024-06-30
7    NIFTY24JULFUT        2024-07-25      Thursday   2024-06-24 to 2024-07-28
8    NIFTY24AUGFUT        2024-08-29      Thursday   2024-07-25 to 2024-09-01
9    NIFTY24SEPFUT        2024-09-26      Thursday   2024-08-25 to 2024-09-29
10   NIFTY24OCT

...

<IPython.core.display.Javascript object>

💾 Saved: MIDCPNIFTY_Futures_2024_2026_Analysis.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


 ✅ COMPLETE!
